## Jupyterhub on Kubernetes

This notebook can be used to launch a standard instance of Jupyterhub which includes a proxy server, and an instance of Jupyterhub. 

Notes on the Server: 
`/etc/jupyterhub/config` is where the configuration is for jupyterhub. 



In [ ]:
#This will import some required libraries.
import sys 
import ruamel.yaml 
#This is your configuration file. 
general_yaml='../../config/config.yaml'
with open(general_yaml, 'r') as yaml:
    cf=ruamel.yaml.round_trip_load(yaml, preserve_quotes=True)

jupyterhub_yaml='../../config/jupyterhub/config_azure_dev.yaml'
with open(jupyterhub_yaml, 'r') as j_yaml:
    cf_j=ruamel.yaml.round_trip_load(j_yaml, preserve_quotes=True)

    #This will allow us to import some useful utilities. 
if cf['docker']:
    cf_j['path']=cf['docker_path']
else:
    cf_j['path']=cf['local_path']

sys.path.append(cf_j['path']+"/lib/kuberutils")   
import importlib
import kuberutils as ku
importlib.reload(ku)
#This will load common commands for your cluster
cf_j=ku.jupyterhub_commands(cf_j)
print(ruamel.yaml.dump(cf_j, sys.stdout, Dumper=ruamel.yaml.RoundTripDumper))

#helm install jupyterhub/jupyterhub \
#    --version=v0.4 \
#    --name=<YOUR-RELEASE-NAME> \
#    --namespace=<YOUR-NAMESPACE> \
#    -f config.yaml

In [ ]:
#This makes a specific directory for configuration
if cf_j['rebuild_config']:
    ku.bash_command('rm -rf '+cf_j['jupyterhub_instance'])
    ku.bash_command('mkdir '+cf_j['jupyterhub_instance'])
    #Random variable that will be added to the config file. 
    ku.bash_command("openssl rand -hex 32 > "+ cf_j['jupyterhub_instance']+"/cookie_secret.txt")
    #Random variable that will be added to the config file.
    ku.bash_command("openssl rand -hex 32 > "+ cf_j['jupyterhub_instance']+"/secret_token.txt")
    #download reference config
    ku.bash_command("wget -P "+ cf_j['jupyterhub_instance']+" "+cf_j['jupyterhub_config_init'])
    ku.init_jupyterhub_config(cf_j)

In [ ]:
print(ku.bash_command('repo_jupyterhub',cf_j))

In [ ]:
#Verify we are messing with the right system. 
print(ku.bash_command("kubectl config current-context"))

In [ ]:
print(ku.bash_command("kubectl get all pods"))

### Helm Versioning.

In order for Helm to work the viersion 

`!helm version` Should yield something like: 
```
Client: &version.Version{SemVer:"v2.5.1", GitCommit:"7cf31e8d9a026287041bae077b09165be247ae66", GitTreeState:"clean"}
Server: &version.Version{SemVer:"v2.5.1", GitCommit:"7cf31e8d9a026287041bae077b09165be247ae66", GitTreeState:"clean"}
```

In [ ]:
!helm version

In [ ]:
#This will install the jupyterhub instance.
print(ku.bash_command('install_jupyterhub',cf_j))

In [ ]:
#This will check on the status of Jupyterhub.
print(ku.bash_command('describe_jupyterhub',cf_j))



In [ ]:
#!kubectl --namespace=kuberlyticsazoct26 logs -f pull-all-nodes-1509221758-kuberlyticsazoct26-1-n6z4l 

In [ ]:
#!kubectl delete --all pods --namespace=kuberlyticsproduction

In [ ]:
!kubectl --namespace=kuberlyticsproduction get svc proxy-public

In [ ]:
cf_j=ku.get_jupyterhub_ip(cf_j)

## Update Authorization
When just starting out, the default configuration launches with no password.  While there is still a username and login screen, 


In [ ]:
#This is how we set the authororization
ku.update_config('auth','github_auth', cf_j)
#ku.update_config('auth','google_auth', cf_j)
#ku.update_config('auth','dummy_auth', cf_j)


In [ ]:
ku.update_config('admin','admin', cf_j)

### Update the Singleuser Container
This can allow you to utilize a different container. 


In [ ]:
ku.update_config('singleuser','singleuser', cf_j)

In [ ]:
#Upgrading Jupyterhub. 
print(ku.bash_command('upgrade_jupyterhub',cf_j))

In [ ]:
cf_k['get_logs']="kubectl --namespace="+cf_j['namespace']+" logs hub-deployment-2149519926-6h09q "
cf_k['get_logs']

In [ ]:
!kubectl --namespace=jupyterhub-dojo logs hub-deployment-3135836638-w63sx

In [ ]:
!kubectl --namespace=jupyterhub-dojo logs hub-deployment-4054782028-0187m

Adding SSL Support via Let's Encrypt
helm repo update 
helm install --name=letsencrypt --namespace=kube-system stable/kube-lego \
             --set config.LEGO_EMAIL=<your-email> \
             --set config.LEGO_URL=https://acme-v01.api.letsencrypt.org/directory

## Cleanup the Installation
This will cleanup the installation, deleting the instance of Jupyterhub.

In [ ]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('delete_jupyterhub',cf_j))

In [ ]:
#Delete Jupyterhub with Helm.
print(ku.bash_command('delete_namespace',cf_j))